<a href="https://colab.research.google.com/github/AAA123404/Financial-big-data-analysis/blob/main/week13%E4%BD%9C%E6%A5%AD_%E5%9F%BA%E5%9B%A0%E6%BC%94%E7%AE%97%E6%B3%95.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [85]:
#pip install yfinance

In [86]:
import yfinance as yf
import pandas as pd
import numpy as np
import random

tsmc_ticker = "2330.TW"
hon_hai_ticker = "2317.TW"
start_date = "2020-01-01"
end_date = "2024-01-01"

tsmc_data = yf.download(tsmc_ticker, start=start_date, end=end_date)
hon_hai_data = yf.download(hon_hai_ticker, start=start_date, end=end_date)

tsmc_data['Daily Return'] = tsmc_data['Adj Close'].pct_change()
hon_hai_data['Daily Return'] = hon_hai_data['Adj Close'].pct_change()


tsmc_mean_return = tsmc_data['Daily Return'].mean()
tsmc_volatility = tsmc_data['Daily Return'].std()
hon_hai_mean_return = hon_hai_data['Daily Return'].mean()
hon_hai_volatility = hon_hai_data['Daily Return'].std()

data = pd.DataFrame({
    '2330.TW': tsmc_data['Daily Return'],
    '2317.TW': hon_hai_data['Daily Return']
})
correlation_matrix = data.corr()



print()
print("台積電平均日收益率:", tsmc_data['Daily Return'].mean())
print("台積電波動率:", tsmc_data['Daily Return'].std())
print("鴻海平均日收益率:", hon_hai_data['Daily Return'].mean())
print("鴻海波動率:", hon_hai_data['Daily Return'].std())
print("相關係數矩陣:")
print(correlation_matrix)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


台積電平均日收益率: 0.0008152954804603258
台積電波動率: 0.017338876880779806
鴻海平均日收益率: 0.0004552035900649189
鴻海波動率: 0.01561363827003127
相關係數矩陣:
         2330.TW  2317.TW
2330.TW  1.00000  0.42758
2317.TW  0.42758  1.00000


In [87]:
def fitness(chromosome):
    tsmc_selected, hon_hai_selected = chromosome

    portfolio_return = (tsmc_mean_return * tsmc_selected +
                        hon_hai_mean_return * hon_hai_selected)
    portfolio_volatility = np.sqrt(
        (tsmc_selected * tsmc_volatility) ** 2 +
        (hon_hai_selected * hon_hai_volatility) ** 2 +
        2 * tsmc_selected * hon_hai_selected *
        tsmc_volatility * hon_hai_volatility *
        correlation_matrix.loc['2330.TW', '2317.TW']
    )

    return portfolio_return / portfolio_volatility if portfolio_volatility > 0 else 0

POPULATION_SIZE = 10
GENERATIONS = 50
MUTATION_RATE = 0.1

#初始化
def initialize_population(size):
    return [np.random.randint(0, 2, 2).tolist() for _ in range(size)]

#選擇
def selection(population, fitness_values):
    total_fitness = sum(fitness_values)
    probabilities = [f / total_fitness for f in fitness_values]
    selected = random.choices(population, probabilities, k=2)
    return selected

#交配
def crossover(parent1, parent2):
    point = random.randint(0, len(parent1) - 1)
    child1 = parent1[:point] + parent2[point:]
    child2 = parent2[:point] + parent1[point:]
    return child1, child2

#突變
def mutate(chromosome):
    for i in range(len(chromosome)):
        if random.random() < MUTATION_RATE:
            chromosome[i] = 1 - chromosome[i]
    return chromosome

population = initialize_population(POPULATION_SIZE)

for generation in range(GENERATIONS):
    fitness_values = [fitness(chromosome) for chromosome in population]

    new_population = []

    while len(new_population) < POPULATION_SIZE:
        parent1, parent2 = selection(population, fitness_values)
        child1, child2 = crossover(parent1, parent2)
        new_population.append(mutate(child1))
        if len(new_population) < POPULATION_SIZE:
            new_population.append(mutate(child2))

    population = new_population

results = []
for chromosome in population:
    tsmc_selected, hon_hai_selected = chromosome
    portfolio_return = (tsmc_mean_return * tsmc_selected +
                        hon_hai_mean_return * hon_hai_selected)
    portfolio_volatility = np.sqrt(
        (tsmc_selected * tsmc_volatility) ** 2 +
        (hon_hai_selected * hon_hai_volatility) ** 2 +
        2 * tsmc_selected * hon_hai_selected *
        tsmc_volatility * hon_hai_volatility *
        correlation_matrix.loc['2330.TW', '2317.TW']
    )
    fitness_value = fitness(chromosome)
    total_selected = tsmc_selected + hon_hai_selected
    tsmc_ratio = tsmc_selected / total_selected if total_selected > 0 else 0
    hon_hai_ratio = hon_hai_selected / total_selected if total_selected > 0 else 0
    results.append((chromosome, tsmc_ratio, hon_hai_ratio, portfolio_return, portfolio_volatility, fitness_value))

best_result = max(results, key=lambda x: x[-1])
chromosome, tsmc_ratio, hon_hai_ratio, portfolio_return, portfolio_volatility, fitness_value = best_result

print(f"最佳投資組合: {chromosome}, 台積電比例: {tsmc_ratio:.2f}, 鴻海比例: {hon_hai_ratio:.2f}, 回報率: {portfolio_return:.4f}, 風險: {portfolio_volatility:.4f}, 適應度值: {fitness_value:.4f}")


最佳投資組合: [1, 0], 台積電比例: 1.00, 鴻海比例: 0.00, 回報率: 0.0008, 風險: 0.0173, 適應度值: 0.0470


In [88]:
def fitness(chromosome):
    tsmc_weight, hon_hai_weight = chromosome


    portfolio_return = (tsmc_mean_return * tsmc_weight +
                        hon_hai_mean_return * hon_hai_weight)
    portfolio_volatility = np.sqrt(
        (tsmc_weight * tsmc_volatility) ** 2 +
        (hon_hai_weight * hon_hai_volatility) ** 2 +
        2 * tsmc_weight * hon_hai_weight *
        tsmc_volatility * hon_hai_volatility *
        correlation_matrix.loc['2330.TW', '2317.TW']
    )

    return portfolio_return / portfolio_volatility if portfolio_volatility > 0 else 0

POPULATION_SIZE = 10
GENERATIONS = 50
MUTATION_RATE = 0.1

#初始化
def initialize_population(size):
    return [np.random.dirichlet(np.ones(2)).tolist() for _ in range(size)]

#選擇
def selection(population, fitness_values):
    total_fitness = sum(fitness_values)
    probabilities = [f / total_fitness for f in fitness_values]
    selected = random.choices(population, probabilities, k=2)
    return selected

#交配
def crossover(parent1, parent2):
    point = random.randint(0, len(parent1) - 1)
    child1 = parent1[:point] + parent2[point:]
    child2 = parent2[:point] + parent1[point:]
    return child1, child2

#突變
def mutate(chromosome):
    for i in range(len(chromosome)):
        if random.random() < MUTATION_RATE:
            chromosome[i] = abs(chromosome[i] + np.random.normal(0, 0.1))
    total = sum(chromosome)
    return [w / total for w in chromosome]


population = initialize_population(POPULATION_SIZE)

for generation in range(GENERATIONS):
    fitness_values = [fitness(chromosome) for chromosome in population]

    new_population = []

    while len(new_population) < POPULATION_SIZE:
        parent1, parent2 = selection(population, fitness_values)
        child1, child2 = crossover(parent1, parent2)
        new_population.append(mutate(child1))
        if len(new_population) < POPULATION_SIZE:
            new_population.append(mutate(child2))

    population = new_population

results = []
for chromosome in population:
    tsmc_weight, hon_hai_weight = chromosome
    portfolio_return = (tsmc_mean_return * tsmc_weight +
                        hon_hai_mean_return * hon_hai_weight)
    portfolio_volatility = np.sqrt(
        (tsmc_weight * tsmc_volatility) ** 2 +
        (hon_hai_weight * hon_hai_volatility) ** 2 +
        2 * tsmc_weight * hon_hai_weight *
        tsmc_volatility * hon_hai_volatility *
        correlation_matrix.loc['2330.TW', '2317.TW']
    )
    fitness_value = fitness(chromosome)
    results.append((chromosome, portfolio_return, portfolio_volatility, fitness_value))


best_result = max(results, key=lambda x: x[-1])
chromosome, portfolio_return, portfolio_volatility, fitness_value = best_result

print(f"最佳投資組合比例: {chromosome}, 回報率: {portfolio_return:.4f}, 風險: {portfolio_volatility:.4f}, 適應度值: {fitness_value:.4f}")

最佳投資組合比例: [0.6431723180066505, 0.3568276819933494], 回報率: 0.0007, 風險: 0.0144, 適應度值: 0.0476
